In [165]:
import sys
import json
import pkg_resources
import types
import os
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np
import re
import datetime
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from bs4 import BeautifulSoup

with open("index.html", encoding="utf-8") as file:
    src = file.read()

soup = BeautifulSoup(src, "lxml")

all_soup = soup.find_all("div", class_='r-table__row')

financing_source_list = []
film_title_list = []
film_studio_list = []
release_date_list = []
refundable_support_list = []
nonrefundable_support_list = []
budget_list = []
box_office_list = []
audience_list = []

for items in all_soup[1:]:
    targets_in_item = items.find_all("div")
    
    try:  
        parse_financing_source = targets_in_item[0]
        parse_financing_source = str(parse_financing_source)
        parse_financing_source = re.findall(r'[А-Яа-я+]+\s[А-Яа-я+]+', parse_financing_source)
        parse_financing_source = ', '.join([str(elem) for elem in parse_financing_source])
        financing_source_list.append(parse_financing_source)
    except: financing_source_list.append('—')
    
    try:
        parse_film_title = targets_in_item[2]
        parse_film_title = parse_film_title.find("div",
                                                 class_="r-table__text r-table__project-name js-bitwise-digit")
        film_title_list.append(parse_film_title.text)
    except: film_title_list.append('—')

    try: 
        parse_film_studio = targets_in_item[5] 
        parse_film_studio = parse_film_studio.find("div",
                                                   class_="r-table__text js-bitwise-digit r-table__company expandable-item visible")
        film_studio_list.append(parse_film_studio.text)
    except: film_studio_list.append('—')
    try: 
        parse_release_date = targets_in_item[7] 
        parse_release_date = parse_release_date.find("div",
                                                   class_="r-table__text js-bitwise-digit")
        release_date_list.append(parse_release_date.text)
    except: release_date_list.append('—')
    try: 
        parse_refundable_support = targets_in_item[9] 
        parse_refundable_support = parse_refundable_support.find("div",
                                                   class_="r-table__text js-bitwise-digit")
        refundable_support_list.append(parse_refundable_support.text)
    except: refundable_support_list.append('—')
    try: 
        parse_nonrefundable_support = targets_in_item[11] 
        parse_nonrefundable_support = parse_nonrefundable_support.find("div",
                                                   class_="r-table__text js-bitwise-digit")
        nonrefundable_support_list.append(parse_nonrefundable_support.text)
    except: nonrefundable_support_list.append('—')
    try: 
        parse_budget = targets_in_item[15] 
        parse_budget = parse_budget.find("div",
                                                   class_="r-table__text js-bitwise-digit")
        budget_list.append(parse_budget.text)
    except: budget_list.append('—')
    try: 
        parse_box_office = targets_in_item[17] 
        parse_box_office = parse_box_office.find("div",
                                                   class_="r-table__text js-bitwise-digit")
        box_office_list.append(parse_box_office.text)
    except: box_office_list.append('—')
    try: 
        parse_audience = targets_in_item[19] 
        parse_audience = parse_audience.find("div",
                                                   class_="r-table__text js-bitwise-digit")
        audience_list.append(parse_audience.text)
    except: audience_list.append('—')
        
        
parsed_data = pd.DataFrame(
    {'financing_source': parse_financing_source_list,
     'title': film_title_list,
     'film_studio': film_studio_list,
          'release_date': release_date_list,
          'refundable_support': refundable_support_list,
          'nonrefundable_support': nonrefundable_support_support_list,
          'budget': budget_list,
          'box_office': box_office_list,
          'audience': audience_list})


for _ in parsed_data.columns.tolist()[4:]:
    parsed_data[_] = parsed_data[_].str.replace(' ','')
    parsed_data[_] = parsed_data[_].str.replace('-','0')
    parsed_data[_] = pd.to_numeric(parsed_data[_], errors='coerce')
    
parsed_data['release_date'] = pd.to_datetime(parsed_data['release_date'], format='%d.%m.%Y', errors='coerce')

In [207]:
import pandas as pd
from datetime import datetime 

parsed_data = pd.DataFrame(
    {'financing_source': parse_financing_source_list,
     'title': film_title_list,
     'film_studio': film_studio_list,
          'release_date': release_date_list,
          'refundable_support': refundable_support_list,
          'nonrefundable_support': nonrefundable_support_support_list,
          'budget': budget_list,
          'box_office': box_office_list,
          'audience': audience_list})


for _ in parsed_data.columns.tolist()[4:]:
    parsed_data[_] = parsed_data[_].str.replace(' ','')
    parsed_data[_] = parsed_data[_].str.replace('-','0')
    parsed_data[_] = pd.to_numeric(parsed_data[_], errors='coerce')
    
parsed_data['release_date'] = pd.to_datetime(parsed_data['release_date'], format='%d.%m.%Y', errors='coerce')

In [208]:
parsed_data.describe(include='all').T

,count,unique,top,freq
financing_source,468,3,Министерство культуры,245
title,468,433,Затмение (Битва с экстрасенсами),4
film_studio,468,215,"ООО ""КИНОКОМПАНИЯ ""СТВ""",24
release_date,468,268,—,11
refundable_support,468,36,-,316
nonrefundable_support,468,87,25 000 000,86
budget,468,355,Данные не предоставлены правообладателем,21
box_office,468,426,—,11
audience,468,425,—,11


In [210]:
parsed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468 entries, 0 to 467
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   financing_source       468 non-null    object        
 1   title                  468 non-null    object        
 2   film_studio            468 non-null    object        
 3   release_date           453 non-null    datetime64[ns]
 4   refundable_support     457 non-null    float64       
 5   nonrefundable_support  457 non-null    float64       
 6   budget                 436 non-null    float64       
 7   box_office             457 non-null    float64       
 8   audience               457 non-null    float64       
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 33.0+ KB


In [211]:
parsed_data.describe(include='all').T

<ipython-input-211-0507ec54962b>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  parsed_data.describe(include='all').T


,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
financing_source,468,3,Министерство культуры,245,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
title,468,433,Затмение (Битва с экстрасенсами),4,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
film_studio,468,215,"ООО ""КИНОКОМПАНИЯ ""СТВ""",24,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
release_date,453,266,2016-03-31 00:00:00,5,2015-01-01,2021-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
refundable_support,457.0,NaN,NaN,NaN,NaT,NaT,11263122.277899,24799503.72864,0.0,0.0,0.0,10000000.0,180000000.0
nonrefundable_support,457.0,NaN,NaN,NaN,NaT,NaT,49442773.339168,66771743.194588,0.0,25000000.0,30000000.0,40000000.0,613000000.0
budget,436.0,NaN,NaN,NaN,NaT,NaT,129348573.983945,186091411.347975,1500000.0,45035439.75,68649916.0,139664936.75,2305074303.0
box_office,457.0,NaN,NaN,NaN,NaT,NaT,127216021.039387,330311565.90925,0.0,596900.0,9197108.0,88363409.0,3074045201.0
audience,457.0,NaN,NaN,NaN,NaT,NaT,519207.085339,1301556.507955,0.0,3649.0,41908.0,382317.0,11903736.0


In [109]:
all_soup = soup.find_all("div", class_='r-table__row')

for items in all_soup[1:]:
    targets_in_item = items.find_all("div")

parse_financing_source_list = []


for el in targets_list:
    _ = targets_list[0] = str(_)
    
    
parse_financing_source_list = []
parse_financing_source = soup.find_all("span", class_='movie-rating__financing-source')

for _ in parse_financing_source:
    _ = str(_)
    _ = re.findall(r'[А-Яа-я+]+\s[А-Яа-я+]+', _)
    _ = ', '.join([str(elem) for elem in _])
    parse_financing_source_list.append(_)
    
len(parse_financing_source_list)

<div class="r-table__col" style="min-width: 100px; max-width: 100px;"><div class="r-table__text r-table__date" x-ms-format-detection="none">1</div><span class="movie-rating__financing-source"><img src="./index_files/financing_source_01.png" title="Фонд кино"/></span></div> 

<div class="r-table__text r-table__date" x-ms-format-detection="none">1</div> 

<div class="r-table__col" style="min-width: 240px; max-width: 240px;"><div class="fixed-column">Холоп</div><div class="r-table__text r-table__project-name js-bitwise-digit"><a href="https://ekinobilet.fond-kino.ru/films/detail/111021719">Холоп</a></div></div> 

<div class="fixed-column">Холоп</div> 

<div class="r-table__text r-table__project-name js-bitwise-digit"><a href="https://ekinobilet.fond-kino.ru/films/detail/111021719">Холоп</a></div> 

<div class="r-table__col" style="min-width: 220px; max-width: 220px;"><div class="r-table__text js-bitwise-digit r-table__company expandable-item visible">ООО "МЕМ"</div></div> 

<div class="r-

In [107]:
len(soup.find_all("div", class_='r-table__row')[0].find_all("div"))

20

In [97]:
soup.find_all("span", class_='movie-rating__financing-source')[289]

<span class="movie-rating__financing-source"><img src="./index_files/financing_source_00.png" title="Министерство культуры"/><img src="./index_files/financing_source_01.png" title="Фонд кино"/></span>

In [90]:
re.findall(r'[А-Яа-я+]+\s[А-Яа-я+]+', str(soup.find_all("span", class_='movie-rating__financing-source')[289]))

['Министерство культуры', 'Фонд кино']

In [93]:
parse_financing_source_list = []
parse_financing_source = soup.find_all("span", class_='movie-rating__financing-source')

for _ in parse_financing_source:
    _ = str(_)
    _ = re.findall(r'[А-Яа-я+]+\s[А-Яа-я+]+', _)
    _ = ', '.join([str(elem) for elem in _])
    parse_financing_source_list.append(_)
    
len(parse_financing_source_list)

468

In [94]:
parse_financing_source_list

['Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Министерство культуры',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Министерство культуры, Фонд кино',
 'Фонд кино',
 'Министерство культуры',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Министерство культуры',
 'Министерство культуры',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Министерство культуры',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Министерство культуры, Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 'Министерство культуры',
 'Фонд кино',
 'Министерство культуры',
 'Фонд кино',
 'Фонд кино',
 'Фонд кино',
 '

In [72]:
film_title_list = []
film_title = soup.find_all("div", class_="r-table__text r-table__project-name js-bitwise-digit")

for _ in film_title:
    film_title_list.append(_.text)
    
len(film_title_list)

468

In [73]:


film_studio_list = []
film_studio = soup.find_all("div", class_="r-table__text js-bitwise-digit r-table__company expandable-item visible")

for _ in film_studio:
    film_studio_list.append(_.text)
    
len(film_studio_list)

479

In [95]:
film_studio_list

['ООО "МЕМ"',
 'ООО "СТУДИЯ "ТРИТЭ" НИКИТЫ МИХАЛКОВА"',
 'ООО "МАРС МЕДИА ЭНТЕРТЕЙНМЕНТ"',
 'АО "ВБД ГРУП"',
 'АО "ВБД ГРУП"',
 'ЗАО "ДИРЕКЦИЯ КИНО"',
 'ООО "ВОДОРОД 2011"',
 'ООО "ВОДОРОД 2011"',
 'ООО "СТУДИЯ "ТРИТЭ" НИКИТЫ МИХАЛКОВА"',
 'ООО "КИНОКОМПАНИЯ "СТВ"',
 'ООО "АРТ ПИКЧЕРС СТУДИЯ"',
 'ООО "АРТ ПИКЧЕРС СТУДИЯ"',
 'ООО "КИНОКОМПАНИЯ "СТВ"',
 'ООО "СТУДИЯ "ТРИТЭ" НИКИТЫ МИХАЛКОВА"',
 'ООО "БАЗЕЛЕВС"',
 'ООО "СТУДИЯ "ТРИТЭ" НИКИТЫ МИХАЛКОВА"',
 'ООО "КИНОСТУДИЯ "СЛОВО"',
 'ООО "КИНОКОМПАНИЯ "СТВ"',
 'ООО "КИНОКОМПАНИЯ "СТВ"',
 'ООО "КИНОКОМПАНИЯ "СТВ"',
 'ООО "БАЗЕЛЕВС"',
 'ЗАО "ДИРЕКЦИЯ КИНО"',
 'ООО "КИНОКОМПАНИЯ "СТВ"',
 'ООО "БАЗЕЛЕВС"',
 'ООО "БАЗЕЛЕВС"',
 'ООО "ДРУГ ДРУГА"',
 'ООО "ЦЕНТРАЛ ПАРТНЕРШИП"',
 'ООО "КИНОКОМПАНИЯ "СТВ"',
 'ООО "БАЗЕЛЕВС"',
 'ООО "НОН-СТОП ПРОДАКШН"',
 'ООО "СТУДИЯ "ТРИТЭ" НИКИТЫ МИХАЛКОВА"',
 'ООО "ТПО "РОК"',
 'ООО "КИНОКОМПАНИЯ "СТВ"',
 'ООО "ЦЕНТРАЛ ПАРТНЕРШИП"',
 'ООО "КОРНЕР ВОРК"',
 'ООО "АРТ ПИКЧЕРС СТУДИЯ"',
 'ООО "ВАЙС ФИЛМС"',
 'ООО "С

In [75]:
release_date_list = []
release_date = soup.find_all("div", class_="r-table__text js-bitwise-digit")

for _ in release_date:
    release_date_list.append(_.text)
    
len(release_date_list)

3276

In [76]:
release_date_list

['26.12.2019',
 '25 000 000',
 '35 000 000',
 '37.5%',
 '160 000 000',
 '3 074 045 201',
 '11 639 702',
 '28.12.2017',
 '-',
 '400 000 000',
 '67.8%',
 '590 192 191',
 '2 943 649 432',
 '11 903 736',
 '01.01.2019',
 '60 000 000',
 '120 000 000',
 '31.9%',
 '563 784 717',
 '2 271 437 355',
 '8 819 706',
 '01.01.2021',
 '100 000 000',
 '100 000 000',
 '30.8%',
 '650 000 000',
 '2 048 383 619',
 '7 147 868',
 '19.10.2017',
 '50 000 000',
 '50 000 000',
 '28.4%',
 '352 477 575',
 '1 660 197 803',
 '7 345 704',
 '29.12.2016',
 '115 000 000',
 '613 000 000',
 '59.4%',
 '1 224 661 889',
 '1 496 865 082',
 '5 649 419',
 '14.02.2020',
 '75 000 000',
 '150 000 000',
 '69.8%',
 '322 350 000',
 '1 469 807 187',
 '6 004 545',
 '14.02.2018',
 '25 000 000',
 '60 000 000',
 '57.2%',
 '148 585 222',
 '1 446 203 938',
 '6 050 287',
 '21.04.2016',
 '50 000 000',
 '392 104 482',
 '69.6%',
 '635 283 867',
 '1 405 080 814',
 '4 990 883',
 '18.02.2021',
 '-',
 '400 000 000',
 '48.7%',
 '821 157 640',
 '1 176

In [69]:
film_studio_list

['ООО "МЕМ"',
 'ООО "СТУДИЯ "ТРИТЭ" НИКИТЫ МИХАЛКОВА"',
 'ООО "МАРС МЕДИА ЭНТЕРТЕЙНМЕНТ"',
 'АО "ВБД ГРУП"',
 'АО "ВБД ГРУП"',
 'ЗАО "ДИРЕКЦИЯ КИНО"',
 'ООО "ВОДОРОД 2011"',
 'ООО "ВОДОРОД 2011"',
 'ООО "СТУДИЯ "ТРИТЭ" НИКИТЫ МИХАЛКОВА"',
 'ООО "КИНОКОМПАНИЯ "СТВ"',
 'ООО "АРТ ПИКЧЕРС СТУДИЯ"',
 'ООО "АРТ ПИКЧЕРС СТУДИЯ"',
 'ООО "КИНОКОМПАНИЯ "СТВ"',
 'ООО "СТУДИЯ "ТРИТЭ" НИКИТЫ МИХАЛКОВА"',
 'ООО "БАЗЕЛЕВС"',
 'ООО "СТУДИЯ "ТРИТЭ" НИКИТЫ МИХАЛКОВА"',
 'ООО "КИНОСТУДИЯ "СЛОВО"',
 'ООО "КИНОКОМПАНИЯ "СТВ"',
 'ООО "КИНОКОМПАНИЯ "СТВ"',
 'ООО "КИНОКОМПАНИЯ "СТВ"',
 'ООО "БАЗЕЛЕВС"',
 'ЗАО "ДИРЕКЦИЯ КИНО"',
 'ООО "КИНОКОМПАНИЯ "СТВ"',
 'ООО "БАЗЕЛЕВС"',
 'ООО "БАЗЕЛЕВС"',
 'ООО "ДРУГ ДРУГА"',
 'ООО "ЦЕНТРАЛ ПАРТНЕРШИП"',
 'ООО "КИНОКОМПАНИЯ "СТВ"',
 'ООО "БАЗЕЛЕВС"',
 'ООО "НОН-СТОП ПРОДАКШН"',
 'ООО "СТУДИЯ "ТРИТЭ" НИКИТЫ МИХАЛКОВА"',
 'ООО "ТПО "РОК"',
 'ООО "КИНОКОМПАНИЯ "СТВ"',
 'ООО "ЦЕНТРАЛ ПАРТНЕРШИП"',
 'ООО "КОРНЕР ВОРК"',
 'ООО "АРТ ПИКЧЕРС СТУДИЯ"',
 'ООО "ВАЙС ФИЛМС"',
 'ООО "С

In [ ]:
links = soup.find(class_="https://ekinobilet.fond-kino.ru/government-support/").find_all("a")

find_all_clothes = soup.find_all(text=re.compile("([Оо]дежда)"))
print(find_all_clothes)

In [ ]:
'puNumber',
'show_start_year',
'show_start_month',
'age_limit',

'title_true',              
'alter_title', 
'title',
                         
'type',
'genres',
'ratings',

'counts_film_studio',
'film_studio',

'counts_production_country',
'production_country',

'counts_director',
'director',

'counts_producer',
'producer',

'financing_source',
'refundable_support',
'nonrefundable_support',
'budget',
'box_office',

'show_start_date',      
'show_start_mnt_yr',
'age_restriction'

In [ ]:


# title = soup.title
# print(title)
# print(title.text)
# print(title.string)

# .find() .find_all()
# page_h1 = soup.find("h1")
# print(page_h1)
#
# page_all_h1 = soup.find_all("h1")
# print(page_all_h1)
#
# for item in page_all_h1:
#     print(item.text)

# user_name = soup.find("div", class_="user__name")
# print(user_name.text.strip())

# user_name = soup.find(class_="user__name").find("span").text
# print(user_name)

# user_name = soup.find("div", {"class": "user__name", "id": "aaa"}).find("span").text
# print(user_name)

# find_all_spans_in_user_info = soup.find(class_="user__info").find_all("span")
# print(find_all_spans_in_user_info)

# for item in find_all_spans_in_user_info:
#     print(item.text)

# print(find_all_spans_in_user_info[0])
# print(find_all_spans_in_user_info[2].text)

# social_links = soup.find(class_="social__networks").find("ul").find_all("a")
# print(social_links)

# all_a = soup.find_all("a")
# print(all_a)
#
# for item in all_a:
#     item_text = item.text
#     item_url = item.get("href")
#     print(f"{item_text}: {item_url}")

# .find_parent() .find_parents()

# post_div = soup.find(class_="post__text").find_parent()
# print(post_div)

# post_div = soup.find(class_="post__text").find_parent("div", "user__post")
# print(post_div)

# post_divs = soup.find(class_="post__text").find_parents("div", "user__post")
# print(post_divs)

# .next_element .previous_element
# next_el = soup.find(class_="post__title").next_element.next_element.text
# print(next_el)
#
# next_el = soup.find(class_="post__title").find_next().text
# print(next_el)

# .find_next_sibling() .find_previous_sibling()
# next_sib = soup.find(class_="post__title").find_next_sibling()
# print(next_sib)

# prev_sib = soup.find(class_="post__date").find_previous_sibling()
# print(prev_sib)

# post_title = soup.find(class_="post__date").find_previous_sibling().find_next().text
# print(post_title)


# print(links)
#
# for link in links:
#     link_href_attr = link.get("href")
#     link_href_attr1 = link["href"]
#
#     link_data_attr = link.get("data-attr")
#     link_data_attr1 = link["data-attr"]
#
#     print(link_href_attr1)
#     print(link_data_attr1)

# find_a_by_text = soup.find("a", text="Одежда")
# print(find_a_by_text)
#
# find_a_by_text = soup.find("a", text="Одежда для взрослых")
# print(find_a_by_text)

# find_a_by_text = soup.find("a", text=re.compile("Одежда"))
# print(find_a_by_text)

